In [ ]:
# Source information: Telegram channels used
channels = [
    "Agdchan",
    "readovkanews",
    "medvedev_telegram",
    "warfakes",
    "MariaVladimirovnaZakharova",
    "alexey_pushkov",
    "RKadyrov_95",
    "holmogor_talks",
    "dva_majors",
    "grey_zone",
    "philologist_zov"
]
print("Data was collected from the following Telegram channels:")
for ch in channels:
    print(f" - @{ch}")

In [ ]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

russian_stopwords = stopwords.words("russian")


In [ ]:
import sys
print(sys.executable)

In [ ]:
import nltk
print(nltk.data.find("tokenizers/punkt"))

In [ ]:
import nltk

nltk.data.path.append(r"C:\Users\yuter\AppData\Roaming\nltk_data")

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
import nltk
print(nltk.data.find("tokenizers/punkt"))

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
print("✅ Все ресурсы NLTK загружены успешно!")

In [ ]:
import spacy

nlp = spacy.load("ru_core_news_md")
print("✅ Русская модель успешно загружена!")

In [ ]:
import re
import spacy
from nltk.corpus import stopwords

nlp = spacy.load("ru_core_news_md")
stop_words = set(stopwords.words("russian"))

def clean_text(text: str) -> str:
    """Очищает текст: удаляет лишнее и приводит к нормальной форме."""
    text = text.lower()  # Приводим к нижнему регистру
    text = re.sub(r"http\S+|www\S+|@\w+|\d+", "", text)  # Удаляем ссылки, упоминания, цифры
    text = re.sub(r"[^\w\s]", "", text)  # Удаляем пунктуацию

    doc = nlp(text)  # Обрабатываем текст в spacy

    words = [token.lemma_ for token in doc if token.text not in stop_words and token.pos_ in {"NOUN", "ADJ"}]  # Лемматизация + фильтр
    return " ".join(words)

# Тест:
example_text = "⚡️В срочном порядке! ВСУ атаковали мирных жителей в Донецке! 📢 Смотрите видео здесь: https://t.me/example"
print(clean_text(example_text))

In [ ]:
import pandas as pd

df = pd.read_csv("bertopic_clustered_results.csv")
df["clean_text"] = df["text"].apply(clean_text)
df.to_csv("cleaned_bertopic_results.csv", index=False)

print("✅ Файл сохранен: cleaned_bertopic_results.csv!")

In [ ]:
from transformers import AutoModel

model_name = "QCRI/PropagandaTechniquesAnalysis-en-BERT"
try:
    model = AutoModel.from_pretrained(model_name)
    print("✅ Модель загружена успешно!")
except Exception as e:
    print(f"❌ Ошибка: {e}")

In [ ]:
from transformers import AutoTokenizer

model_name = "facebook/bart-large-mnli"  # ← Попробуй другие модели
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("✅ Tokenizer загружен успешно!")

In [ ]:
pipe = pipeline("text-classification", model="DeepPavlov/rubert-base-cased")  # Русский BERT

In [ ]:
import os
os.environ["GIT_PYTHON_GIT_EXECUTABLE"] = "C:\\Program Files\\Git\\bin\\git.exe"

In [ ]:
print(dataset)
print(dataset.column_names)
print(dataset["train"][0])  # Выведем первый пример из train

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# 📌 1. Загружаем токенизатор и модель
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

# 📌 2. Создаём список всех уникальных меток и словарь преобразования
label_list = sorted(set(label for labels in dataset["train"]["labels"] for label in (labels if isinstance(labels, list) else [labels])))
label_to_id = {label: i for i, label in enumerate(label_list)}  # Создаём словарь соответствий

# 📌 3. Загружаем модель с учётом многоклассовой классификации
model = AutoModelForSequenceClassification.from_pretrained(
    "DeepPavlov/rubert-base-cased",
    num_labels=len(label_list)  # Указываем количество классов
)

# 📌 4. Функция токенизации + преобразование списка меток в индексы
def preprocess_function(example):
    return {
        **tokenizer(example["text"], padding="max_length", truncation=True, max_length=512),
        "labels": [label_to_id[label] for label in example["labels"]]  # Преобразуем метки в индексы
    }

# 📌 5. Обрабатываем датасет
train_dataset = dataset["train"].map(preprocess_function, batched=True)
test_dataset = dataset["test"].map(preprocess_function, batched=True)

# 📌 6. Настройки тренировки
training_args = TrainingArguments(
    output_dir="./fallacy_bert",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01
)

# 📌 7. Запуск тренировки
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

In [ ]:
metrics = trainer.evaluate()
print(metrics)

In [ ]:
trainer.save_model("./fallacy_bert")  # Сохраняем модель

In [ ]:
import os

print(os.listdir("./fallacy_bert"))

In [ ]:
import psutil

for proc in psutil.process_iter(['pid', 'name']):
    try:
        for item in proc.open_files():
            if './fallacy_bert' in item.path:
                print(f"Процесс {proc.info['name']} (PID: {proc.info['pid']}) использует файл {item.path}")
    except psutil.AccessDenied:
        pass

In [ ]:
label_list = sorted(set(label for example in dataset["train"] for label in (example["labels"] if isinstance(example["labels"], list) else [example["labels"]])))
id_to_label = {i: label for i, label in enumerate(label_list)}

In [ ]:
decoded_predictions = [
    {"label": id_to_label[int(pred["label"].split("_")[-1])], "score": pred["score"]}
    for pred in predictions
]

print(decoded_predictions)

In [ ]:
label_list = list(set(dataset["train"]["labels"]))  # Собираем все уникальные метки
id_to_label = {i: label for i, label in enumerate(label_list)}

In [ ]:
print("🔹 Загруженные метки:", label_list)

In [ ]:
train_labels = dataset["train"]["labels"]  # Используем весь список меток
label_distribution = Counter([label for sublist in train_labels for label in sublist])  # Собираем статистику

In [ ]:
print("🔹 Пример данных:", train_labels[:5])  # Показываем первые 5 записей

In [ ]:
from collections import Counter

# Создаём список предсказанных меток
predicted_labels = [pred["label"] for pred in decoded_predictions]

# Подсчитываем количество встречаемости каждой ошибки
fallacy_counts = Counter(predicted_labels)

print("🔹 Частота ошибок:", fallacy_counts)

In [ ]:
print(predictions[:10])  # Проверяем первые 10 предсказаний

In [ ]:
decoded_predictions = [
    {"label": id_to_label.get(int(pred["label"].split("_")[-1]), "UNKNOWN"), "score": pred["score"]}
    for pred in predictions
]

In [ ]:
fallacy_counts = Counter([pred["label"] for pred in decoded_predictions])
print("🔹 Исправленный список ошибок:", fallacy_counts)

In [ ]:
predicted_labels = [pred["label"] for pred in decoded_predictions]
fallacy_counts = Counter(predicted_labels)

print("🔹 Частота ошибок в предсказаниях:", fallacy_counts)

In [ ]:
from collections import Counter

train_labels = [label for example in dataset["train"]["labels"] for label in example]
label_distribution = Counter(train_labels)

print("🔹 Частота ошибок в обучающем датасете:", label_distribution)

In [ ]:
label_list = sorted(set(label for example in dataset["train"] for label in (example["labels"] if isinstance(example["labels"], list) else [example["labels"]])))
id_to_label = {i: label for i, label in enumerate(label_list)}

In [ ]:
fallacy_counts = Counter([pred["label"] for pred in decoded_predictions])
print("🔹 Исправленное распределение ошибок:", fallacy_counts)

In [ ]:
print([pred["label"] for pred in predictions[:20]])  # Проверяем первые 20 предсказаний

In [ ]:
print(decoded_predictions[:20])  # Проверяем первые 20 преобразованных предсказаний

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./fallacy_bert")
token_lengths = [len(tokenizer.tokenize(text)) for text in texts]

print(f"🔹 Средняя длина токенизированных текстов: {sum(token_lengths) / len(token_lengths):.2f}")
print(f"🔹 Максимальная длина текста: {max(token_lengths)}")

In [ ]:
print("🔹 Все возможные метки:", id_to_label)

In [ ]:
print("🔹 Размер обучающего датасета:", len(dataset["train"]))

In [ ]:
scores = [pred["score"] for pred in decoded_predictions]
print(f"🔹 Средний score: {sum(scores) / len(scores):.4f}")
print(f"🔹 Максимальный score: {max(scores)}")
print(f"🔹 Минимальный score: {min(scores)}")

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "bert-base-multilingual-cased"  # Выбираем предобученную BERT-модель
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_list))
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
train_dataset = dataset["train"]  # Просто используем train из dataset
train_dataset = train_dataset.map(tokenize_function)

In [ ]:
print(type(dataset["train"]))  # Должно вывести <class 'datasets.Dataset'>

In [ ]:
training_args = TrainingArguments(
    output_dir="./bert_finetuned",
    eval_strategy="no",  # Отключаем eval_strategy, если нет eval_dataset
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)

In [ ]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

train_dataset = dataset["train"]
train_dataset = train_dataset.map(tokenize_function)

# Проверяем, есть ли метки
print(train_dataset.column_names)  # Должно быть "labels"

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

In [ ]:
for key in ["input_ids", "attention_mask", "labels"]:
    print(f"🔹 {key}: {type(batch[key])}")

In [ ]:
label_to_id = {v: k for k, v in id_to_label.items()}  # Переворачиваем словарь
train_dataset = train_dataset.map(lambda example: {"labels": label_to_id[example["labels"]]})

In [ ]:
print(train_dataset[0]["labels"])  # Должно вывести число (например, 3, 7, 10)

In [ ]:
print(model.config)  # Должно быть что-то вроде `num_labels=13`

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=len(label_list)  # Задаём количество классов ошибок
)

In [ ]:
print(model.config.num_labels)  # Должно вывести `13` (или другое количество классов)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=len(label_list)  # Указываем число классов
)

In [ ]:
label_to_id = {v: k for k, v in id_to_label.items()}  # Переворачиваем словарь
train_dataset = train_dataset.map(lambda example: {"labels": label_to_id.get(example["labels"], -1)})  # Используем .get(), чтобы избежать KeyError

In [ ]:
print(set(example["labels"] for example in train_dataset))

In [ ]:
train_dataset = train_dataset.map(lambda example: {"labels": label_to_id.get(example["labels"], "UNKNOWN")})

In [ ]:
print(train_dataset.column_names)  # Должно включать "labels"

In [ ]:
train_dataset = train_dataset.map(lambda example: {"labels": label_to_id.get(example["labels"], -1)})

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

In [ ]:
print(model.config)  # Должно включать "num_labels"

In [ ]:
train_dataset = train_dataset.filter(lambda example: example["labels"] in label_to_id.values())

In [ ]:
print("🔹 label_to_id:", label_to_id)
print("🔹 Метки в train_dataset:", set(example["labels"] for example in dataset["train"]))

In [ ]:
train_dataset = train_dataset.filter(lambda example: example["labels"] != -1)

In [ ]:
for example in train_dataset:
    if example["labels"] == -1:
        print("❌ Ошибка: метка -1 найдена!", example)

In [ ]:
count_invalid = sum(1 for example in train_dataset if example["labels"] == -1)
print(f"❌ Осталось {count_invalid} примеров с -1")

In [ ]:
train_dataset = train_dataset.map(lambda example: {"labels": max(example["labels"], 0)})  # -1 → 0

In [ ]:
print(set(example["labels"] for example in train_dataset))

In [ ]:
train_dataset = train_dataset.map(lambda example: {"labels": label_to_id.get(example["labels"], 0)})  # `-1` → `0`

In [ ]:
train_dataset = dataset["train"].map(lambda example: {"labels": label_to_id.get(example["labels"], 0), **example})

In [ ]:
print(label_to_id)

In [ ]:
count_invalid = sum(1 for example in train_dataset if example["labels"] == -1)
print(f"❌ Осталось {count_invalid} примеров с -1")

In [ ]:
print(label_to_id)  # Должен содержать {'ad hominem': 0, 'false dilemma': 1, ...}
print(set(example["labels"] for example in train_dataset))  # Должны быть только числа от `0` до `num_labels - 1`

In [ ]:
model.config.num_labels = len(label_to_id)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

In [ ]:
train_dataset = train_dataset.map(lambda example: {"labels": label_to_id[example["labels"]], **example})

In [ ]:
def convert_labels(example):
    if example["labels"] in label_to_id:
        new_label = label_to_id[example["labels"]]
        print(f"✅ Конвертация: {example['labels']} → {new_label}")  # Выводим замену
        return {"labels": new_label, **example}
    else:
        print(f"❌ Ошибка: {example['labels']} не найдено в `label_to_id`")
        return example  # Оставляем без изменений, чтобы увидеть, что происходит

train_dataset = train_dataset.map(convert_labels)

In [ ]:
from datasets import DatasetDict

dataset_new = DatasetDict({
    "train": train_dataset.map(lambda example: {"labels": label_to_id[example["labels"]], **example})
})

print(set(example["labels"] for example in dataset_new["train"]))  # Должны быть только числа

In [ ]:
from datasets import Value

# ШАГ 1: Создаём новый столбец с числовыми метками.
def convert_labels(example):
    # Если метка хранится в списке, возьмём первый элемент (или продумайте логику для множества меток)
    label_str = example["labels"][0] if isinstance(example["labels"], list) else example["labels"]
    # Возвращаем новый столбец "int_labels" с числовым значением
    return {"int_labels": label_to_id[label_str]}

# Применяем map и записываем в новый датасет
train_dataset_new = train_dataset.map(convert_labels, batched=False)

# ШАГ 2: Удаляем старый столбец "labels" и переименовываем "int_labels" в "labels"
train_dataset_new = train_dataset_new.remove_columns("labels")
train_dataset_new = train_dataset_new.rename_column("int_labels", "labels")

# ШАГ 3: Приводим столбец "labels" к типу int (int64)
train_dataset_new = train_dataset_new.cast_column("labels", Value("int64"))

# Проверяем, что в датасете теперь только числа
print(set(example["labels"] for example in train_dataset_new))

In [ ]:
def tokenize_function(example):
    # Здесь можно настроить truncation, padding и max_length по необходимости.
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

# Применяем токенизацию ко всему датасету.
tokenized_dataset = train_dataset.map(tokenize_function, batched=True)

In [ ]:
def convert_labels(example):
    # Если метка оказалась списком – берём первый элемент
    label_str = example["labels"][0] if isinstance(example["labels"], list) else example["labels"]
    return {"int_labels": label_to_id[label_str]}

# Применяем map для создания нового числового столбца
tokenized_dataset = tokenized_dataset.map(convert_labels)

In [ ]:
# Это должно вывести соответствующие числовые значения, например, {0, 1, ...}
print(set(example["int_labels"] for example in tokenized_dataset))

In [ ]:
# Удаляем старую колонку "labels"
tokenized_dataset = tokenized_dataset.remove_columns("labels")
# Переименовываем "int_labels" → "labels"
tokenized_dataset = tokenized_dataset.rename_column("int_labels", "labels")

In [ ]:
from datasets import Value
tokenized_dataset = tokenized_dataset.cast_column("labels", Value("int64"))

In [ ]:
for example in tokenized_dataset.select(range(5)):
    print(type(example["labels"]), example["labels"])  # Должны быть int

In [ ]:
from datasets import Value
from transformers import AutoTokenizer

# Пример словаря меток
label_to_id = {
    'ad hominem': 0,
    'ad populum': 1,
    'appeal to emotion': 2,
    'circular claim': 3,
    'deductive fallacy': 4,
    'equivocation': 5,
    'fallacy of credibility': 6,
    'fallacy of extension': 7,
    'fallacy of relevance': 8,
    'false causality': 9,
    'false dilemma': 10,
    'faulty generalization': 11,
    'intentional': 12,
}

# Инициализируем токенизатор (замените на нужную модель)
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# --- Шаг 1. Токенизация текста ---
def tokenize_function(example):
    # Токенизируем текст: включаем truncation, padding и задаём max_length по необходимости
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

# Применяем токенизацию ко всему датасету (предполагается, что train_dataset уже загружен)
tokenized_dataset = train_dataset.map(tokenize_function, batched=True)

# --- Шаг 2. Преобразование меток ---
def convert_labels(example):
    # Если labels — это список, возьмем первый элемент
    label_value = example["labels"]
    if isinstance(label_value, list):
        label_value = label_value[0]
    # Преобразуем строку метки в число с помощью словаря label_to_id
    return {"int_labels": label_to_id[label_value]}

# Создаем новый столбец "int_labels" с числовыми значениями
tokenized_dataset = tokenized_dataset.map(convert_labels)

# --- Шаг 3. Удаление оригинального столбца и переименование ---
# Удаляем старую колонку с метками (строковой)
tokenized_dataset = tokenized_dataset.remove_columns("labels")
# Переименовываем "int_labels" в "labels"
tokenized_dataset = tokenized_dataset.rename_column("int_labels", "labels")

# --- Шаг 4. Приведение типа столбца ---
# Приводим колонку "labels" к типу int64
tokenized_dataset = tokenized_dataset.cast_column("labels", Value("int64"))

# --- Шаг 5. Установка формата для PyTorch ---
# Задаем список колонок, которые необходимы модели.
columns_to_use = ["input_ids", "attention_mask", "labels"]
# Если токенизатор возвращает token_type_ids — добавим сюда
if "token_type_ids" in tokenized_dataset.column_names:
    columns_to_use.append("token_type_ids")

# Очень важно: вызываем set_format без переприсваивания!
tokenized_dataset.set_format("torch", columns=columns_to_use)

# --- Проверка схемы ---
print("Column names:", tokenized_dataset.column_names)
print("Features:", tokenized_dataset.features)

In [ ]:
# 1. Импорты
from datasets import load_dataset, Value
from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

# 2. Загрузка CSV-файлов с данными
data_files = {
    "train": "https://huggingface.co/datasets/benmshultz/RuFal_fallacy_detection/resolve/main/fallacies_ru_train.csv",
    "eval": "https://huggingface.co/datasets/benmshultz/RuFal_fallacy_detection/resolve/main/fallacies_ru_test.csv"
}
raw_datasets = load_dataset("csv", data_files=data_files)

# 3. Определение словаря меток (label_to_id)
label_to_id = {
    'ad hominem': 0,
    'ad populum': 1,
    'appeal to emotion': 2,
    'circular claim': 3,
    'deductive fallacy': 4,
    'equivocation': 5,
    'fallacy of credibility': 6,
    'fallacy of extension': 7,
    'fallacy of relevance': 8,
    'false causality': 9,
    'false dilemma': 10,
    'faulty generalization': 11,
    'intentional': 12,
}

# 4. Инициализация токенизатора (русскоязычный)
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

# 5. Функция токенизации
def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

# Токенизируем тренировочный и eval наборы
tokenized_train = raw_datasets["train"].map(tokenize_function, batched=True)
tokenized_eval = raw_datasets["eval"].map(tokenize_function, batched=True)

# 6. Функция преобразования меток из строк в числа
def convert_labels(example):
    label_val = example["labels"]
    if isinstance(label_val, list):
        label_val = label_val[0]
    return {"int_labels": label_to_id[label_val]}

tokenized_train = tokenized_train.map(convert_labels)
tokenized_eval = tokenized_eval.map(convert_labels)

# 7. Удаляем исходный столбец "labels" (строкового типа) и переименовываем новый числовой столбец
tokenized_train = tokenized_train.remove_columns("labels")
tokenized_train = tokenized_train.rename_column("int_labels", "labels")

tokenized_eval = tokenized_eval.remove_columns("labels")
tokenized_eval = tokenized_eval.rename_column("int_labels", "labels")

# 8. Приводим колонку "labels" к типу int64
tokenized_train = tokenized_train.cast_column("labels", Value("int64"))
tokenized_eval = tokenized_eval.cast_column("labels", Value("int64"))

# 9. Устанавливаем формат датасета для PyTorch (метод set_format мутирует датасет "на месте")
columns_to_use = ["input_ids", "attention_mask", "labels"]
if "token_type_ids" in tokenized_train.column_names:
    columns_to_use.append("token_type_ids")
tokenized_train.set_format("torch", columns=columns_to_use)
tokenized_eval.set_format("torch", columns=columns_to_use)

# Проверка схемы, можно вывести:
print("Train columns:", tokenized_train.column_names)
print("Train features:", tokenized_train.features)
print("Eval columns:", tokenized_eval.column_names)
print("Eval features:", tokenized_eval.features)

# 10. Инициализируем модель
model = BertForSequenceClassification.from_pretrained(
    "DeepPavlov/rubert-base-cased",
    num_labels=len(label_to_id)
)

# 11. Определяем аргументы обучения (legacy-параметры для старых версий transformers)
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",      # Для новых версий это evaluation_strategy
    logging_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    save_steps=2000,
    eval_steps=10,
    learning_rate=2e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

# 12. Создаем Trainer, передаём в него тренировочную и eval выборки
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval
)

# 13. Запускаем обучение
trainer.train()

In [ ]:
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")

In [ ]:
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer, BertForSequenceClassification

# 1. Определение словаря меток (label_to_id)
label_to_id = {
    'ad hominem': 0,
    'ad populum': 1,
    'appeal to emotion': 2,
    'circular claim': 3,
    'deductive fallacy': 4,
    'equivocation': 5,
    'fallacy of credibility': 6,
    'fallacy of extension': 7,
    'fallacy of relevance': 8,
    'false causality': 9,
    'false dilemma': 10,
    'faulty generalization': 11,
    'intentional': 12,
}

# Инвертированный словарь для преобразования id в название логической ошибки:
id_to_label = {v: k for k, v in label_to_id.items()}

# 2. Загрузка модели и токенизатора
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = BertForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased", num_labels=len(label_to_id))
model.eval()  # перевод модели в режим оценки

# Определяем устройство (GPU, если доступно)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 3. Загрузка датасета с текстами
df = pd.read_csv("cleaned_bertopic_results.csv")

# 4. Агрегация softmax‑вероятностей по всем постам
num_labels = len(label_to_id)
aggregate_probs = np.zeros(num_labels, dtype=np.float64)

with torch.no_grad():
    for text in df["text"]:
        # Токенизация текста (усечение, паддинг до max_length=128)
        inputs = tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=128,
            return_tensors="pt"
        )
        # Переносим данные на устройство модели (GPU или CPU)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = model(**inputs)
        logits = outputs.logits  # размер [1, num_labels]
        probabilities = torch.softmax(logits, dim=-1).squeeze(0)  # размер [num_labels]
        aggregate_probs += probabilities.cpu().numpy()

# 5. Нормализация: получаем доли от суммы
total = aggregate_probs.sum()
distribution = aggregate_probs / total

# 6. Подготовка меток для графика (сортировка по порядку id)
labels = [id_to_label[i] for i in range(num_labels)]

# 7. Построение графика распределения
plt.figure(figsize=(12, 6))
sns.barplot(x=labels, y=distribution, palette="viridis")
plt.xticks(rotation=45, ha="right")
plt.xlabel("Тип логической ошибки")
plt.ylabel("Доля (нормированная сумма softmax вероятностей)")
plt.title("Агрегированное распределение логических ошибок по всему датасету")
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd

# Создаем DataFrame с fallacy и соответствующей долей
df_distribution = pd.DataFrame({'Fallacy': labels, 'Distribution': distribution})
# Сортируем по столбцу Distribution от большего к меньшему
df_distribution = df_distribution.sort_values(by='Distribution', ascending=False)

# Построение графика
plt.figure(figsize=(12, 6))
sns.barplot(data=df_distribution, x='Fallacy', y='Distribution', palette='viridis')
plt.xticks(rotation=45, ha="right")
plt.xlabel("Logical Fallacy")
plt.ylabel("Normalized Sum of Softmax Probabilities")
plt.title("Aggregated Distribution of Logical Fallacies Across the Dataset")
plt.tight_layout()
plt.show()